In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [26]:
import time
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import pandas as pd
import json
from torch.utils.data import DataLoader
from IPython.display import clear_output
from IPython.display import display
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"    # 不加这句下面的跑不出来


In [25]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

#### **Easy way** 

In [4]:
loader = DataLoader(train_set, batch_size=len(train_set), num_workers=1)
data = next(iter(loader))
data[0].mean(), data[0].std()

data[0].shape

torch.Size([60000, 1, 28, 28])

#### **Harder way**

In [5]:
loader = DataLoader(train_set, batch_size=1000, num_workers=1)
num_of_pixels = len(train_set) * 28 * 28

In [6]:
total_sum = 0
for batch in loader:
    total_sum += batch[0].sum()
mean = total_sum / num_of_pixels
sum_of_squared_error = 0
for batch in loader:
    sum_of_squared_error += ((batch[0] - mean).pow(2)).sum()
std = torch.sqrt(sum_of_squared_error / num_of_pixels)

mean, std

(tensor(0.2860), tensor(0.3530))

In [7]:
data[0].shape

torch.Size([60000, 1, 28, 28])

In [8]:
# import matplotlib.pyplot as plt

# plt.hist(data[0].flatten())
# plt.axvline(data[0].mean())

In [20]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.fc1(t.reshape(-1, 12 * 4 * 4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [36]:
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = 0
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
        
    def begin_run(self, run ,network, loader):
        self.run_start_time = time.time()
            
        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter()
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        self.tb.add_image('image', grid)
        self.tb.add_graph(self.network, images.to(getattr(run, 'device', 'cpu')))
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0
        
    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
    
    def end_epoch(self):
        
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)
        
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
        
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        
        for k, v in self.run_params._asdict().items():
            results[k] = v
            
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns') 
        
        clear_output(wait=True)
        display(df)
        
    def track_loss(self, loss):
        self.epoch_loss += loss.item() * self.loader.batch_size 
    
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)   
        
    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self, fileName):
        
        pd.DataFrame.from_dict(
            self.run_data,
            orient="columns"
        ).to_csv(f'{fileName}.csv')
        
        with open(f'{fileName}.json', 'w', encoding='utf-8')as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)
               
        

In [37]:
class RunBuilder():
    @staticmethod   # 无需创建实例，可通过指定类进行调用的方法
    def get_runs(params):
        Run = namedtuple('Run', params.keys())
        runs = []
        
        for v in product(*params.values()):
            runs.append(Run(*v))
            
        return runs

#### **Using the** *mean* **and** *std* **Values** 

In [38]:
train_set_normal = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)

In [39]:
loader = DataLoader(train_set_normal, batch_size=len(train_set), num_workers=1)
data = next(iter(loader))
data[0].mean(), data[0].std()

(tensor(-1.8842e-07), tensor(1.))

In [40]:
trainsets = {
    'not_normal': train_set,
    'normal': train_set_normal
}

In [41]:
params = OrderedDict(
    lr = [.01],
    batch_size = [1000],
    num_workers = [1],
    device = ['cuda'],
    trainset = ['not_normal', 'normal']
)

m = RunManager()
for run in RunBuilder.get_runs(params):
    
    device = torch.device(run.device)
    network = Network().to(device)
    loader = DataLoader(trainsets[run.trainset], batch_size=run.batch_size, num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    for epoch in range(20):
        m.begin_epoch()
        for batch in loader:
            
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds, labels)
            
        m.end_epoch()
    m.end_run()
    
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset
0,1,1,0.970909,0.625500,6.274126,15.345351,0.01,1000,1,cuda,not_normal
1,1,2,0.517372,0.801083,5.894568,21.428761,0.01,1000,1,cuda,not_normal
2,1,3,0.418177,0.844933,6.045565,27.558579,0.01,1000,1,cuda,not_normal
3,1,4,0.370213,0.863850,5.684935,33.296366,0.01,1000,1,cuda,not_normal
4,1,5,0.340086,0.874100,5.653236,39.009773,0.01,1000,1,cuda,not_normal
5,1,6,0.314214,0.882300,5.763992,44.834790,0.01,1000,1,cuda,not_normal
6,1,7,0.300537,0.887633,5.756770,50.646873,0.01,1000,1,cuda,not_normal
7,1,8,0.286131,0.891617,5.713441,56.420379,0.01,1000,1,cuda,not_normal
8,1,9,0.271522,0.898533,5.917674,62.406709,0.01,1000,1,cuda,not_normal
9,1,10,0.259518,0.903000,6.354302,68.824546,0.01,1000,1,cuda,not_normal


In [42]:
pd.DataFrame.from_dict(m.run_data).sort_values('accuracy', ascending=False)

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset
39,2,20,0.211679,0.920083,7.731518,160.124770,0.01,1000,1,cuda,normal
38,2,19,0.215057,0.919033,7.548729,152.335628,0.01,1000,1,cuda,normal
19,1,20,0.214996,0.918450,5.818768,129.581353,0.01,1000,1,cuda,not_normal
37,2,18,0.221612,0.917367,7.611170,144.724444,0.01,1000,1,cuda,normal
18,1,19,0.219283,0.916617,5.791958,123.707492,0.01,1000,1,cuda,not_normal
17,1,18,0.220838,0.915717,6.007973,117.863940,0.01,1000,1,cuda,not_normal
16,1,17,0.227880,0.913900,5.704103,111.798518,0.01,1000,1,cuda,not_normal
14,1,15,0.227762,0.913250,5.713966,100.115401,0.01,1000,1,cuda,not_normal
15,1,16,0.228855,0.912633,5.821826,106.009526,0.01,1000,1,cuda,not_normal
36,2,17,0.232848,0.912450,7.666469,137.063150,0.01,1000,1,cuda,normal
